# **CamemBERT**

[CamemBERT](https://camembert-model.fr/) est un modèle de traitement automatique de la langue Française, basé sur l'architecture [RoBERTa](https://ai.facebook.com/blog/roberta-an-optimized-method-for-pretraining-self-supervised-nlp-systems/) développée par Facebook AI en 2019, dédiée à l'Anglais. RoBERTa est lui même basé sur [BERT](https://fr.wikipedia.org/wiki/BERT_(mod%C3%A8le_de_langage)) qui a été développé par Google en 2018.  
CamemBERT est donc un cousin Français de BERT, qui a pu voir le jour lorsque les équipes de Facebook associés aux chercheurs de [l'INRIA](https://www.inria.fr/fr) ont rendu public ce modèle pré-entrainé sur 138GB de texte Français.  
CamemBERT a été pré-entraîné sur un corpus francophone et avec des hyper-paramètres différents découverts et testés pour la première fois par l’équipe de Facebook. Le choix de ces hyper-paramètres était tellement réussi que l’entreprise a annoncé le sortie d’un “nouveau” modèle baptisé RoBERTa. Pourtant, il n’y a rien de nouveau dans RoBERTA qui comme CamemBERT reste une copie de BERT. Voici ces hyper-paramètres:  
*  CamemBERT choisit les mots à prédire de manière dynamique, c’est-à-dire, non pas lors du pré-processing des données en entrée, mais lors de forward pass, en masquant au hasard certains mots d’une séquence.  
*  Il utilise un batch size différent: ~8 000 contre 256 dans le cas de BERT.  
*  CamemBERT a un seul objectif de pré-entrainement: prédiction des “mots masqués” d’une séquence. BERT en avait deux : prédiction des “mots masqués” et de la phrase suivante d’une séquence. Ce dernier objectif s’est avéré improductif pour l’entrainement.

# **Exemple d'utlisation de CamemBERT**

CamemBERT a été entrainé dans le but de prédire des "mots masqués" dans un texte. Nous allons voir un exemple de ce que peut faire ce modèle pré-entrainé.

Commençons par installer sur la machine les modules python dont nous aurons besoin et qui ne sont pas pré-installés. Ensuite nous importons les bibliothèques.  
En particulier, nous utiliserons la bibliothèque [Transformers](https://huggingface.co/transformers/#) créée par [Hugging Face](https://huggingface.co/). Cette bibliothèque contient des centaines de modèles pré-entrainés pour réaliser des opérations sur les données textuelles, comme la classification, l'extraction d'informations, le "questions-réponses", la traduction, ...

In [1]:
!pip install transformers --quiet

     |████████████████████████████████| 890kB 3.0MB/s 
     |████████████████████████████████| 890kB 15.9MB/s 
     |████████████████████████████████| 1.1MB 13.3MB/s 
     |████████████████████████████████| 3.0MB 25.0MB/s 


In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

from transformers import TFCamembertForMaskedLM
from transformers import AutoTokenizer

Définisson la variable tokenizer qui permettra d'instancier le tokenizer pour CamemBERT fourni par la librairie "transformers" :

In [21]:
tokenizer = AutoTokenizer.from_pretrained('jplu/tf-camembert-base',)

Définissons maintenant la phrase que nopus souhaitons compléter puis utlisons le tokenizer de la bibliothèque "transformers" pour préparer le texte :

In [22]:
phrase = "L'intelligence artificielle va mener à la <mask> du monde !"
output_tokenizer = tokenizer.encode_plus(phrase, max_length=100, padding="longest", truncation=True, return_tensors='tf')
output_tokenizer

{'input_ids': <tf.Tensor: shape=(1, 14), dtype=int32, numpy=
array([[    5,    71,    11,  6031,  7956,   198,  3532,    15,    13,
        32004,    25,   164,    83,     6]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 14), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

Récupérons la position du masque (mask) dans la séquence retournée par le tokenizer :

In [29]:
mask_index = (output_tokenizer['input_ids'][0].numpy() == tokenizer.mask_token_id).nonzero()
mask_index = np.reshape(mask_index,(1))[0]
mask_index

9

Instancions maintenant le modèle CamemBERT pré-entrainé avec le jeu de données de base pour Tensorflow :

In [27]:
model = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")

All model checkpoint weights were used when initializing TFCamembertForMaskedLM.

All the weights of TFCamembertForMaskedLM were initialized from the model checkpoint at jplu/tf-camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForMaskedLM for predictions without further training.


Puis lançons le modèle sur la phrase :

In [38]:
output = model(output_tokenizer['input_ids'])[0]
output

<tf.Tensor: shape=(1, 14, 32005), dtype=float32, numpy=
array([[[ 23.108753  ,  -3.755758  ,   8.674742  , ...,  -5.096422  ,
          -4.313377  ,   2.3206162 ],
        [ -2.5845957 ,  -3.958022  ,  14.981613  , ...,  -7.429527  ,
          -1.1563265 ,  -3.0757904 ],
        [  1.1122129 ,  -7.8049626 ,   4.1219034 , ..., -12.544399  ,
          -7.452759  ,  -4.76741   ],
        ...,
        [  4.594441  ,  -8.668797  ,   9.463923  , ..., -14.600243  ,
         -14.187639  ,   0.7320218 ],
        [  3.9691653 ,  -8.347343  ,  11.592593  , ...,  -9.434902  ,
          -9.269459  ,  -0.75447655],
        [  9.361578  ,  -4.3270473 ,  27.006542  , ...,  -6.9380293 ,
          -6.461198  ,   3.1999168 ]]], dtype=float32)>

On récupère les (32005 !) valeurs en sortie du modèle correspondants à l'emplacement du masque :

In [39]:
output  = output[0, mask_index, :]
output

<tf.Tensor: shape=(32005,), dtype=float32, numpy=
array([ 1.0191815, -2.3711014,  1.6898267, ..., -8.149946 , -5.941703 ,
       -2.0453548], dtype=float32)>

Puis on applique une fonctioon d'activation Soft-Max afin de normaliser les probabilités sur chaque valeurs :

In [40]:
proba = tf.nn.softmax(output)
proba

<tf.Tensor: shape=(32005,), dtype=float32, numpy=
array([2.1088415e-08, 7.1066097e-10, 4.1238351e-08, ..., 2.1975666e-12,
       1.9997254e-11, 9.8431119e-10], dtype=float32)>

On récupère ensuite les valeurs et les indices des 5 plus grandes probabilités parmi ces 32005 probabilités :

In [42]:
top_proba, top_indices = tf.math.top_k(proba,k=5)
print(top_proba)
print(top_indices)

tf.Tensor([0.6222108  0.08322746 0.05718182 0.03852326 0.02997597], shape=(5,), dtype=float32)
tf.Tensor([ 259 9408 1691 8710 5876], shape=(5,), dtype=int32)


On va chercher ensuite les valeurs numériques des mots correspondants aux emplacements de ces probibilités les plus fortes, puis on reconvertit ces valeurs en mots réels (fonction inverse du tokenizer) :

In [43]:
topk_predicted_token_bpe2 = " ".join([tokenizer.convert_ids_to_tokens(int(tf.keras.backend.get_value(top_indices[i]))) for i in range(len(top_indices))])
topk_predicted_token_bpe2 

'▁fin ▁conquête ▁découverte ▁domination ▁destruction'

On place dans la variable "mask" le mot clé utlisé dans la phrase pour le masque :

In [44]:
mask = tokenizer.mask_token
mask

'<mask>'

On sépare les résultats obtenus :

In [45]:
topk_predicted_token_bpe2.split(" ")

['▁fin', '▁conquête', '▁découverte', '▁domination', '▁destruction']

Puis on remplace le masque dans la phrase initiale, en utlisant toutes les possibilités trouvées :

In [46]:
topk_filled_outputs_ = []
for index2, predicted_token_bpe2 in enumerate(topk_predicted_token_bpe2.split(" ")):
  predicted_token_ = predicted_token_bpe2.replace("\u2581", " ")
  if " {0}".format(tokenizer.mask_token) in phrase:
    topk_filled_outputs_.append((phrase.replace(" {0}".format(tokenizer.mask_token), predicted_token_)))
  else:
    topk_filled_outputs_.append((phrase.replace(tokenizer.mask_token, predicted_token_)))

topk_filled_outputs_

["L'intelligence artificielle va mener à la fin du monde !",
 "L'intelligence artificielle va mener à la conquête du monde !",
 "L'intelligence artificielle va mener à la découverte du monde !",
 "L'intelligence artificielle va mener à la domination du monde !",
 "L'intelligence artificielle va mener à la destruction du monde !"]